In [1]:
! pip install lineapy

DEPRECATION: Loading egg at /home/arthur/anaconda3/lib/python3.11/site-packages/vUnet_AFM-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.3/209.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.8/563.8 kB 11.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: mistune
    Found existing installation: mistune 2.0.4
    Uninstalling mistune-2.0.4:
      Successfully uninstalled mistune-2.0.4
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.25
    Uninstalling SQLAlche

In [2]:
! python -m pip install pandas==1.3.2

DEPRECATION: Loading egg at /home/arthur/anaconda3/lib/python3.11/site-packages/vUnet_AFM-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [867 lines of output]
      Ignoring numpy: markers 'python_version == "3.7" and (platform_machine != "arm64" or platform_system != "Darwin") and platform_machine != "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.8" and (platform_machine != "arm64" or platform_system != "Darwin") and platform_machine != "aarch64"' don't match your environment
      Ignoring numpy: markers 'p

In [3]:
%load_ext lineapy

Folder /home/arthur/.lineapy does not exist. Creating a new one.
Folder /home/arthur/.lineapy/linea_pickles does not exist. Creating a new one.
Folder /home/arthur/.lineapy/custom-annotations does not exist. Creating a new one.


In [4]:
lineapy.options

lineapy_config(home_dir=PosixPath('/home/arthur/.lineapy'), database_url='sqlite:////home/arthur/.lineapy/db.sqlite', artifact_storage_dir=PosixPath('/home/arthur/.lineapy/linea_pickles'), customized_annotation_folder=PosixPath('/home/arthur/.lineapy/custom-annotations'), do_not_track=False, logging_level='INFO', logging_file=PosixPath('/home/arthur/.lineapy/lineapy.log'), storage_options=None, mlflow_registry_uri=None, mlflow_tracking_uri=None, default_ml_models_storage_backend=None)

In [5]:
import lineapy
import joblib
import json
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Load dataset

In [6]:
# Get data 

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
dataset = data.frame
dataset.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
# print labels for target values 

[print(f'{target}: {label}') for target, label in zip(data.target.unique(), data.target_names)]

IndexError: -1

In [ ]:
# feature names

dataset.columns = [colname.strip(' (cm)').replace(' ', '_') for colname in dataset.columns.tolist()]

feature_names = dataset.columns.tolist()[:4]
feature_names

In [ ]:
#save raw data as artifact
dataset_csv = './data/raw/iris.csv'
dataset.to_csv(dataset_csv, index=False)


In [ ]:
dataset

In [ ]:
print(pd.__version__)

In [ ]:
#save raw data as artifact to lineapy
lineapy.save(dataset, "iris-raw")

# Features engineering

In [ ]:
dataset['sepal_length_to_sepal_width'] = dataset['sepal_length'] / dataset['sepal_width']
dataset['petal_length_to_petal_width'] = dataset['petal_length'] / dataset['petal_width']

dataset = dataset[[
    'sepal_length', 'sepal_width', 'petal_length', 'petal_width',
#     'sepal_length_in_square', 'sepal_width_in_square', 'petal_length_in_square', 'petal_width_in_square',
    'sepal_length_to_sepal_width', 'petal_length_to_petal_width',
    'target'
]]

In [ ]:
dataset.head()

In [ ]:
# Save features
features_path = './data/processed/featured_iris.csv'
dataset.to_csv(features_path, index=False)

In [ ]:
#save features to lineapy
lineapy.save(dataset, "iris-preprocessed")

# Split dataset

In [ ]:
test_size=0.2

## Splittail train/test

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)
train_dataset.shape, test_dataset.shape

In [ ]:
# Save train and test sets
trainset_path = './data/processed/train_iris.csv'
testset_path = './data/processed/test_iris.csv'

train_dataset.to_csv(trainset_path)
test_dataset.to_csv(testset_path)

In [ ]:
#save train and test sets to lineapy
lineapy.save(train_dataset, "train-dataset")
lineapy.save(test_dataset, "test-dataset")

# Train

In [ ]:
# Get X and Y

y_train = train_dataset.loc[:, 'target'].values.astype('int32')
X_train = train_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
# Create an instance of Logistic Regression Classifier CV and fit the data

logreg = LogisticRegression(C=0.001, solver='lbfgs', multi_class='multinomial', max_iter=100)
logreg.fit(X_train, y_train)

In [ ]:
model_path= './models/model.joblib'
joblib.dump(logreg, model_path)

In [ ]:
#save model to lineapy
lineapy.save(model_path, "logreg-model")

# Evaluate

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    
    return plt.gcf()

In [ ]:
# Get X and Y

y_test = test_dataset.loc[:, 'target'].values.astype('int32')
X_test = test_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
prediction = logreg.predict(X_test)
cm = confusion_matrix(prediction, y_test)
f1 = f1_score(y_true = y_test, y_pred = prediction, average='macro')

In [ ]:
# f1 score value
f1

In [ ]:
# Save metrics
metrics_file = './reports/metrics.json'

metrics = {
    'f1': f1
}

with open(metrics_file, 'w') as mf:
    json.dump(
        obj=metrics,
        fp=mf,
        indent=4
    )


In [ ]:
cm_plot = plot_confusion_matrix(cm, data.target_names, normalize=False)

In [ ]:
# Save confusion matrix image
confusion_matrix_image = './reports/confusion_matrix.png'
cm_plot.savefig(confusion_matrix_image)

In [ ]:
#save confusion matrix to lineapy
lineapy.save(plot_confusion_matrix, "plot-confusion-matrix")

In [ ]:
#commenting for change
